# Channels' tags network

In [146]:
import os
import shlex
import json
import csv
import nltk
import itertools
import networkx as nx
import pandas as pd
import collections

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

### Tag network using .csv file

In [147]:
def create_tag_network(fileName, quantile, g, limit):
    j=0
    lemmatizer = WordNetLemmatizer()
    df=pd.read_csv(fileName)
    df = df[df.views>quantile]
    df=df.sort_values(by=['views'],ascending=False)

    for n in range(0,len(df)):
        all_keys=[] # all tag/keywords for each channel without repetition
        try:
            if(df.keywords[n]!=""):

                try:
                    for kw in shlex.split(df.keywords[n]):
                        kw_id = kw.lower().replace("'", "").replace("´", "").replace("’", "")
                        single_keys=nltk.word_tokenize(kw_id)
                        for key in single_keys:
                            key2=lemmatizer.lemmatize(key)
                            if key2 not in all_keys:
                                all_keys.append(key2)

                except:
                    for kw in df.keywords[n].split():
                        kw_id = kw.lower().replace("'", "").replace("´", "").replace("’", "")
                        single_keys=nltk.word_tokenize(kw_id)
                        for key in single_keys:
                            key2=lemmatizer.lemmatize(key)
                            if key2 not in all_keys:
                                all_keys.append(key2)
                
                #remove english stopwords and add to graph                
                for key3 in all_keys:
                    if key3 in stopwords.words('english'):
                        all_keys.remove(key3)
                    else:
                        if g.has_node(key3):
                            g.node[key3]['weight']+=1
                        else:
                            g.add_node(key3, type='keyword', weight=1)
                        
                #add all edges (all keywords appearing together will be connected)
                edges= itertools.combinations(all_keys,2)
                
                for edge in edges:
                    #if the edge exist add +1 weight
                    if g.has_edge(edge[0],edge[1]):
                        g[edge[0]][edge[1]]['weight'] += 1
                    else:
                        g.add_edge(edge[0],edge[1], weight=1)            
                
#                 filter just first limited number of channels
#                 j+=1
#                 if (j==limit):
#                     break
        except:
            continue       

### Get degree quantile to filter information

In [148]:
def get_quantile(q_edge, gf):
    #find maximum and minimum degree 
    degrees = [val for (node, val) in gf.degree()]
    ctr=collections.Counter(degrees)
    #find third quartile in degree
    df2 = pd.DataFrame.from_dict(ctr,orient='index').reset_index()
    q=df2.quantile(q_edge)
    return float(q.values[1])

## Merge nodes 

In [149]:
def merge_nodes(new_node, node_list, gf):
    H =gf.copy()
    gf.add_node(new_node, type='keyword') # Add the 'merged' node   

    for n1,n2,data in H.edges(data=True):
        # For all edges related to one of the nodes to merge,
        # make an edge going to or coming from the `new gene`.
        if n1 in node_list:
            g.add_edge(new_node,n2)
        elif n2 in node_list:
            g.add_edge(n1,new_node)

    for n in node_list: # remove the merged nodes
        gf.remove_node(n)

### Create network using file name

In [150]:
fileNames=['minecraft', 'overwatch','agario', 'LOL', 'callOD','fivenaf','pkgo','roblox', 'gta','happyW']
data_folder_channels='/media/aruiz/data/channels_clean_data/'
fileName2=os.path.join(data_folder_channels,'all_channels_clean.csv')
g = nx.Graph()
create_tag_network(fileName2, 42261, g, 400)       

In [74]:
H =g.copy()
i=0
for node,d in H.nodes(data=True):
    try:
        if (d['weight']<3):
            g.remove_node(node)
    except:
        i+=1
        continue
#     if g.degree(node)< q:
#         g.remove_node(node)

In [153]:
print(len(g))
q=get_quantile(0.75,g)
print(q)

17605
8.0


In [291]:
#minecraft --- node merging
merge_nodes('lets play',['let','play'],g)
merge_nodes('gta V',['gta','v'],g)
merge_nodes('call of duty',['call','duty'],g)

In [80]:
nx.write_gexf(g, "testL.gexf")

In [152]:
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
H=g.copy()

for node, d in H.nodes(data=True):
    try:
        if node in english_vocab:
            g.remove_node(node)
    except:
        continue

In [101]:
# df=pd.read_csv(fileName)
w=0
H=g.copy()
for node, d in H.nodes(data=True):
    try:
#         if node in list_nodes:
#             print(d['weight'])
        if(d['weight']<2):
            g.remove_node(node)
    except:
        continue

In [ ]:
#Number of tags 23373
#17605
# 25% correctly classified as english
# 75% classified